In [1]:
import tensorflow as tf
from tensorflow.contrib import layers
import numpy as np
import math

In [6]:

#         norm = layers.batch_norm if i > 0 else None
#         n_channels = min(256, 16 * 2 ** i)

#         img = layers.conv2d(img,
#                   n_channels, 
#                   scope='disc_'+str(i), 
#                   kernel_size=5, 
#                   activation_fn=lrelu, 
#                   stride=1,
#                   normalizer_fn=norm, 
#                   weights_initializer=initializer)

class Net(object):
    def __init__(self, img_size, n_classes):
        with tf.variable_scope('net'):
            n_layers = int(math.log(img_size) / math.log(2))

            inputs = tf.placeholder(tf.float32, [None, img_size, img_size, 3], name='inputs')
            labels = tf.placeholder(tf.int64, [None], name='labels')

            initial_chans = 8
            x = self.conv(inputs, initial_chans, 1)
            print x.get_shape()

            for i, layer in enumerate(xrange(n_layers)):
                x = self.inception_module(x, initial_chans * 2 ** (i+1), 'inception-{}'.format(i))
                x = layers.batch_norm(x)
                x = self.pool(x, 2, 2)
                print x.get_shape()

            logits = self.conv(x, n_classes, 1, activation=tf.identity)
            logits = tf.reshape(logits, [-1, n_classes])

            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
            predictions = tf.argmax(logits, axis=-1)
            accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

            global_step = tf.contrib.framework.get_or_create_global_step()
            lr = tf.placeholder(tf.float32, name='lr')
            train_op = tf.train.AdamOptimizer(lr).minimize(loss, global_step=global_step)

            self.inputs = inputs
            self.labels = labels
            self.loss = loss
            self.predictions = predictions
            self.accuracy = accuracy
            self.global_step = global_step
            self.train_op = train_op
    
    def conv(self, x, channels, ksize, activation=tf.nn.relu):
        return layers.conv2d(x, 
                             channels, 
                             kernel_size=ksize, 
                             activation_fn=activation)
    
    def pool(self, x, ksize, stride, type='max'):
        fn = {'max': tf.nn.max_pool, 'avg': tf.nn.avg_pool}[type]
        return fn(x, [1, ksize, ksize, 1], [1, stride, stride, 1], 'SAME')
    
    def inception_module(self, x, out_channels, name):
        # https://culurciello.github.io/tech/2016/06/04/nets.html
        # out_channels channels must be a multiple of 4
        
        with tf.variable_scope(name):
            channels = x.get_shape()[-1].value
            bsize = out_channels / 4
            
            y1 = self.conv(x, bsize, 1)
            y2 = self.conv(self.pool(x, 3, 1, 'avg'), bsize, 1)
            y3 = self.conv(self.conv(x, bsize, 1), bsize, 3)
            y4 = self.conv(self.conv(self.conv(x, bsize, 1), bsize, 3), bsize, 3)
            
            return tf.concat([y1, y2, y3, y4], axis=-1)

Net(128, 10)
print 'ok'

(?, 128, 128, 8)
(?, 64, 64, 16)
(?, 32, 32, 32)
(?, 16, 16, 64)
(?, 8, 8, 128)
(?, 4, 4, 256)
(?, 2, 2, 512)
(?, 1, 1, 1024)
ok


## Inception module v3 

<img src='https://culurciello.github.io/assets/nets/inceptionv3.jpg' width=300 />